In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(
  temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=120,
  return_messages=True
)

examples = [
  {
    "question": "탑건",
    "answer": "🛩️🧑‍✈️🔥"
  },
  {
    "question": "대부",
    "answer": "🧑‍🧑‍🧒🔫🍝"
  },
  {
    "question": "괴물",
    "answer": "🦖🏢👣"
  },
  {
    "question": "미션임파서블",
    "answer": "🕶️💥🔒"
  }
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  suffix="Human: {question}",
  input_variables=["question"]
)

def load_memory(input):
  return memory.load_memory_variables({})["history"] # 메모리에 저장되어 있는 내용을 불러옴

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
  result = chain.invoke({"question": question})
  memory.save_context({"input": question}, {"output": result.content}) # 메모리에 저장
  print(result)

In [14]:
invoke_chain("괴물")

content='AI: 🦑🌊🚢'


In [15]:
invoke_chain("파묘")

content='AI: 🐱🔍🔦'


In [16]:
load_memory({})

[HumanMessage(content='괴물'),
 AIMessage(content='AI: 🦑🌊🚢'),
 HumanMessage(content='파묘'),
 AIMessage(content='AI: 🐱🔍🔦')]

In [17]:
invoke_chain("미션임파서블")

content='AI: 🕶️💥🔒'


In [18]:
invoke_chain("겨울왕국")

content='AI: ❄️👑🏰'


In [19]:
load_memory({})

[HumanMessage(content='괴물'),
 AIMessage(content='AI: 🦑🌊🚢'),
 HumanMessage(content='파묘'),
 AIMessage(content='AI: 🐱🔍🔦'),
 HumanMessage(content='미션임파서블'),
 AIMessage(content='AI: 🕶️💥🔒'),
 HumanMessage(content='겨울왕국'),
 AIMessage(content='AI: ❄️👑🏰')]